# Simplified English Word Validator using DFA

This notebook demonstrates how to build and visualize a **Deterministic Finite Automaton (DFA)** that accepts simplified English words.  
A word is accepted if:
- It is **non-empty**
- It contains **only lowercase English letters** (`a–z`)


In [ ]:
# Installation of required packages
!pip install visual-automata
!pip install automathon coloraide pygraphviz
!apt-get install -y graphviz libgraphviz-dev
!pip install pygraphviz

## DFA Logic

We define a DFA with three states:
- `start`: Initial state
- `valid`: Accepting state
- `dead`: Rejecting state

Transitions:
- From `start` to `valid` on `[a–z]`
- From `valid` to itself on `[a–z]`
- Any other character leads to `dead`


In [ ]:
# Word Validator function

def validate_word(word: str) -> str:
    current_state = "start"

    for character in word:
        if current_state == "start":
            if character.islower():
                current_state = "valid"
            else:
                return "Not Accepted"
        elif current_state == "valid":
            if character.islower():
                continue
            else:
                return "Not Accepted"

    return "Accepted" if current_state == "valid" else "Not Accepted"


def run_validator():
    print("Simplified English Word Validator (DFA)")
    print("Type 'exit' to stop.\n")

    while True:
        input_word = input("Enter a word: ")
        if input_word.lower() == "exit":
            print("Exiting the validator. Goodbye!")
            break

        result = validate_word(input_word)
        print(f"Result: {result}\n")


if __name__ == "__main__":
    run_validator()

## DFA Visualization

We use `networkx` and `matplotlib` to draw the DFA graphically.  
The accepting state (`valid`) is shown with a double circle.  
Transitions are labeled with `[a–z]` or `other` for invalid characters.


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, FancyArrow
from visual_automata.fa.dfa import DFA

def visualize_simplified_word_dfa(figsize=(8,4)):
    G = nx.MultiDiGraph()
    states = ["start", "valid", "dead"]
    G.add_nodes_from(states)

    # We'll label transitions compactly: [a-z] and other
    G.add_edge("start", "valid", label="[a-z]")
    G.add_edge("start", "dead",  label="other (Σ \\ [a-z])")
    G.add_edge("valid", "valid", label="[a-z]")
    G.add_edge("valid", "dead",  label="other (Σ \\ [a-z])")
    G.add_edge("dead", "dead",   label="any")

    # Nice fixed positions
    pos = {"start": (-1.2, 0.4), "valid": (0.8, 0.4), "dead": (-0.2, -0.6)}

    plt.figure(figsize=figsize)
    ax = plt.gca()
    ax.set_title("DFA for simplified English words\n(accepts non-empty strings of [a–z])")

    # draw nodes
    nx.draw_networkx_nodes(G, pos, node_size=2200, node_color="#e3f2fd", edgecolors="k")
    nx.draw_networkx_labels(G, pos, font_size=11, font_weight="bold")

    # edges
    nx.draw_networkx_edges(G, pos, connectionstyle="arc3, rad=0.12", arrowsize=20)
    edge_labels = nx.get_edge_attributes(G, "label")
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=10, label_pos=0.5)

    # initial arrow
    start_pos = pos["start"]
    arrow = FancyArrow(start_pos[0]-0.4, start_pos[1], 0.28, 0.0,
                       width=0.02, length_includes_head=True, head_width=0.08, head_length=0.05, color="k")
    ax.add_patch(arrow)

    # double circle for accepting state 'valid'
    valid_pos = pos["valid"]
    outer = Circle(valid_pos, 0.28, fill=False, linewidth=1.6, zorder=3)
    inner = Circle(valid_pos, 0.22, fill=False, linewidth=1.6, zorder=3)
    ax.add_patch(outer)
    ax.add_patch(inner)

    # dead state single circle drawn already by nodes; optionally mark 'dead' label smaller
    ax.text(0.9, -0.9, "Legend:\n[a-z] = lowercase letters\nother = digits, spaces, uppercase, punctuation",
            fontsize=9, bbox=dict(boxstyle="round", fc="wheat", alpha=0.9))

    ax.set_axis_off()
    plt.show()

# call the function to display
visualize_simplified_word_dfa()

import string
def validate_word(word: str) -> str:
    if not word:
        return "Not Accepted"
    if word[0] not in string.ascii_lowercase:
        return "Not Accepted"
    for ch in word[1:]:
        if ch not in string.ascii_lowercase:
            return "Not Accepted"
    return "Accepted"

# Example batch test
samples = ["cat","dog","a","zebra","dog1","1dog","DogHouse","Dog_house"," cats",""]
for s in samples:
    print(f"{s!r:12} -> {validate_word(s)}")